In [141]:
import gzip
import math
import matplotlib.pyplot as plt
import numpy
import random
import sklearn
import string
from collections import defaultdict
from gensim.models import Word2Vec
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/jpuray/nltk_data'
    - '/Users/jpuray/venv/nltk_data'
    - '/Users/jpuray/venv/share/nltk_data'
    - '/Users/jpuray/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [91]:
import pandas as pd
import requests
import json
import pickle

pd.set_option('display.max_rows', None)

In [92]:
#Reads in file

f = open("Ads5000", "rb")
ads = pickle.load(f)
f.close() 

In [93]:
df = pd.DataFrame(ads)
df.head()

,impressions,demographic_distribution,ad_creation_time,ad_delivery_start_time,delivery_by_region,estimated_audience_size,page_name,publisher_platforms,spend,id,ad_creative_link_description,ad_delivery_stop_time
0,"{'lower_bound': '1000', 'upper_bound': '1999'}","[{'percentage': '0.004557', 'age': '18-24', 'g...",2022-01-25,2022-01-25,"[{'percentage': '0.016578', 'region': 'Alabama...",{'lower_bound': '1000001'},MomsRising.org,"[facebook, instagram]","{'lower_bound': '0', 'upper_bound': '99'}",510093387113722,NaN,NaN
1,"{'lower_bound': '2000', 'upper_bound': '2999'}","[{'percentage': '0.000519', 'age': '25-34', 'g...",2022-01-25,2022-01-25,"[{'percentage': '0.026604', 'region': 'Alabama...",{'lower_bound': '1000001'},CPAC 2022,[facebook],"{'lower_bound': '0', 'upper_bound': '99'}",1010286123213947,NaN,NaN
2,"{'lower_bound': '0', 'upper_bound': '999'}",NaN,2022-01-25,2022-01-25,NaN,"{'lower_bound': '10001', 'upper_bound': '50000'}",David Livingston Scott County Magistrate- 7th ...,[facebook],"{'lower_bound': '0', 'upper_bound': '99'}",726465978738772,NaN,NaN
3,"{'lower_bound': '0', 'upper_bound': '999'}","[{'percentage': '0.003436', 'age': '18-24', 'g...",2022-01-25,2022-01-25,"[{'percentage': '1', 'region': 'Florida'}]",{'lower_bound': '1000001'},Miami's Community Newspapers,"[facebook, instagram]","{'lower_bound': '0', 'upper_bound': '99'}",1182151668859503,NaN,NaN
4,"{'lower_bound': '0', 'upper_bound': '999'}","[{'percentage': '0.021875', 'age': '25-34', 'g...",2022-01-21,2022-01-25,"[{'percentage': '0.025316', 'region': 'Alabama...",{'lower_bound': '1000001'},Pew Research Center,"[facebook, instagram]","{'lower_bound': '0', 'upper_bound': '99'}",466037478231880,NaN,NaN


In [94]:
# drops all ads with no description and turns it back into json 

df_with_only_descriptions = df.dropna()
df_json = df_with_only_descriptions.to_json(orient='records')
dataset = json.loads(df_json)

In [95]:
# Removes capitallization / punctuations

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset:
    r = ''.join([c for c in d['ad_creative_link_description'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1
len(wordCount)

4779

In [96]:
# stores an array of tuples with the number of times 
 # each word appears across all the ads 
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [109]:
word_list = [x[1] for x in counts]
len(word_list)

4779

In [98]:
# document frequency 

df = defaultdict(int)
for d in dataset:
    r = ''.join([c for c in d['ad_creative_link_description'].lower() if not c in punctuation])
    for w in set(r.split()):
        df[w] += 1

In [99]:
# term frequency & TF - IDF
tf = defaultdict(int)
for d in dataset:
    r = ''.join([c for c in d['ad_creative_link_description'].lower() if not c in punctuation])
    for w in r.split():
        # Note = rather than +=, different versions of tf could be used instead
        tf[w] = 1
    
tfidf = dict(zip(words,[tf[w] * math.log2(len(dataset) / df[w]) for w in words]))
tfidfQuery = [tf[w] * math.log2(len(dataset) / df[w]) for w in words]

In [100]:
# Finds the highest tf-idf words in our example review

maxTf = [(tf[w],w) for w in words]
maxTf.sort(reverse=True)
maxTfIdf = [(tfidf[w],w) for w in words]
maxTfIdf.sort(reverse=True)

In [101]:
maxTfIdf[:10]

[(10.615629636968098, '😘'),
 (10.615629636968098, '💉'),
 (10.615629636968098, '✔'),
 (10.615629636968098, '✅no'),
 (10.615629636968098, '✅january'),
 (10.615629636968098, '✅free'),
 (10.615629636968098, '✅food'),
 (10.615629636968098, '→'),
 (10.615629636968098, '№21–22–17'),
 (10.615629636968098, '…on')]

In [102]:
maxTfIdf[:-10:-1]

[(0.9151899188270054, 'to'),
 (1.0046048396607452, 'the'),
 (1.6184501560304763, 'and'),
 (1.6269449501959317, 'in'),
 (1.8674367873786373, 'a'),
 (2.1197746100809267, 'for'),
 (2.1521052636969173, 'of'),
 (2.26690148273702, 'now'),
 (2.391627962769993, 'covid19')]